In [1]:
import numpy as np
import pandas as pd
import os
import torch

In [4]:
train_file = pd.read_json('train/RE/RE/RE-train.json')
select_file = pd.read_json('train/RE/RE/train-select.json')

In [2]:
task = 'RE'
dataset = 'RE'

In [5]:
ppl_array = np.zeros(len(train_file))
for process_num in range(1,9,1): ## maximum of k process
    if os.path.exists('ppl/{}/{}/ppl-init-{}.csv'.format(task,dataset,process_num)): ## i-th gradient 
        ppl_df = pd.read_csv('ppl/{}/{}/ppl-init-{}.csv'.format(task,dataset,process_num),index_col=0)
        for index,row in ppl_df.iterrows():
            ppl_array[index] = row[0]

/tmp/ipykernel_926606/459085310.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ppl_array[index] = row[0]
/tmp/ipykernel_926606/459085310.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ppl_array[index] = row[0]
/tmp/ipykernel_926606/459085310.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ppl_array[index] = row[0]
/tmp/ipykernel_926606/459085310.py:6: FutureWarning: Series.__getitem__ treating keys as positions is depr

- `ppl_array`: shape和文件全集保持相同，所有数据by case的perplexity number，一个array
- `ppl_array`越大越困难，data selection baseline可以按照ppl_array

- `greedyList_All_norm_flatten`是一个dict，key是index，value是对应的Facility Location Score
- 需要写一个函数，来检查哪些更大
- `greedyList_All_norm_flatten`越大越好

In [6]:
task = 'RE'
dataset = 'RE'
import torch
import numpy as np
greedyList_All_norm_flatten = torch.load('selection/{}/{}/FL-Score.pkl'.format(task,dataset),weights_only=False)

In [9]:
np.array(list(greedyList_All_norm_flatten.values())).max()

nan

In [8]:
batch_sampler = torch.load('Influence/{}/{}/batch.pkl'.format(task,dataset),weights_only=False)
sample_IF = torch.load('Influence/{}/{}/score.pkl'.format(task,dataset),weights_only=False)

In [9]:
def z_score_normalize(sample_IF):
    """
    对sample_IF字典中各个'method'对应的值（键为index、值为float的字典）进行Z-score归一化。

    参数:
    sample_IF (dict): 包含多个'method'键的字典，每个'method'键对应的值为需要进行归一化处理的字典数据。

    返回:
    dict: 归一化后的字典，结构与输入的sample_IF一致，其中每个'method'键对应的值都已经完成Z-score归一化。
    """
    for method in sample_IF.keys():
        # 获取当前method对应需要归一化的值列表，保持原有顺序
        values_list = list(sample_IF[method].values())
        # 将列表转换为torch.Tensor
        tensor_value = torch.tensor(values_list).unsqueeze(1)  # 添加维度，变为二维张量

        # 计算均值和标准差
        mean_value = tensor_value.mean()
        std_value = tensor_value.std()

        # 进行Z-score归一化
        normalized_tensor = (tensor_value - mean_value) / std_value

        # 将归一化后的结果再转换回列表
        normalized_list = normalized_tensor.squeeze(1).tolist()

        # 更新原字典中当前method对应的值
        index_list = list(sample_IF[method].keys())
        normalized_dict = {}
        for index, normalized_value in zip(index_list, normalized_list):
            normalized_dict[index] = normalized_value

        sample_IF[method] = normalized_dict

    return sample_IF

In [10]:
sample_IF = z_score_normalize(sample_IF)

In [11]:
sample_IF['iterative']

{133: -5.140116214752197,
 134: -5.140116214752197,
 143: -5.140116214752197,
 145: -5.140116214752197,
 149: -5.140116214752197,
 3142: -4.857326984405518,
 3196: -4.857326984405518,
 3058: -3.6425840854644775,
 3084: -3.6425840854644775,
 3096: -3.6425840854644775,
 3104: -3.6425840854644775,
 3130: -3.6425840854644775,
 3171: -3.6425840854644775,
 3177: -3.6425840854644775,
 6400: -3.477116584777832,
 6403: -3.477116584777832,
 6405: -3.477116584777832,
 6408: -3.477116584777832,
 6410: -3.477116584777832,
 6413: -3.477116584777832,
 6420: -3.477116584777832,
 6421: -3.477116584777832,
 1874: -3.4581925868988037,
 1875: -3.4581925868988037,
 1877: -3.4581925868988037,
 1879: -3.4581925868988037,
 1880: -3.4581925868988037,
 1881: -3.4581925868988037,
 1882: -3.4581925868988037,
 1883: -3.4581925868988037,
 6402: -3.388976573944092,
 6407: -3.388976573944092,
 6414: -3.388976573944092,
 6422: -3.388976573944092,
 6426: -3.388976573944092,
 6431: -3.388976573944092,
 6433: -3.38897657

In [37]:
np.argsort(ppl_array)[-10:]

array([1667,  942, 1678, 5770, 1673, 1674, 1677, 1669, 5234, 6461])

In [38]:
ppl_array[6461]

63.75

In [26]:
import heapq
def get_top_k_indices(data_dict: dict, k: int, IF = False) -> list:
    """
    从字典中获取前 K 个最大数值对应的索引。

    Args:
        data_dict (dict): 输入字典，key 为索引，value 为数值。
                          例如：{0: 10, 1: 5, 2: 20, 3: 8}
        k (int): 要获取的最大元素的数量。

    Returns:
        list: 包含前 K 个最大数值对应索引的列表，按数值从大到小排序。
              如果 k 大于字典的元素数量，将返回所有索引。
    """
    if not isinstance(data_dict, dict):
        raise TypeError("Input 'data_dict' must be a dictionary.")
    if not isinstance(k, int) or k < 0:
        raise ValueError("Input 'k' must be a non-negative integer.")

    if k == 0:
        return []
    if not data_dict:
        return []

    # 使用 heapq.nlargest 获取前 K 个最大的 (value, key) 对
    # nlargest 默认按第一个元素（value）排序
    # 注意：nlargest 返回的顺序是数值从大到小
    if not IF: ## 如果按照降序排列，默认为True
        top_k_items = heapq.nlargest(k, data_dict.items(), key=lambda item: item[1])
    else: ## 如果按照升序排列
        top_k_items = heapq.nsmallest(k, data_dict.items(), key=lambda item: item[1])        

    # 提取索引（key）并返回
    # item[0] 是原始的 key
    top_k_indices = [item[0] for item in top_k_items]

    return top_k_indices


In [33]:
sample_IF['proposed'][3142]

-4.491122722625732

- IF=True: 倒序查找，IF
- IF=False: 顺序查找

In [ ]:
get_top_k_indices(sample_IF['proposed'],k=len(select_file),IF=False) 

### Checking Metrics

{'ppl': {'micro_f1': 0.8912733446519525, 'macro_f1': 0.7014128856663336},
 'FL': {'micro_f1': 0.9272665534804754, 'macro_f1': 0.7401582353894606},
 'IF': {'micro_f1': 0.7860780984719864, 'macro_f1': 0.5283008735899821}}